##  Transfer Learning for Flower Species Classification using TensorFlow & Python

### Step 1: Import Libraries and Load Dataset:

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the flower dataset from TensorFlow Datasets
dataset_name = 'tf_flowers'
(ds_train, ds_validation), ds_info = tfds.load(
    dataset_name, split=['train[:80%]', 'train[80%:]'],
    as_supervised=True, with_info=True
)

# Print dataset information
print("Dataset Info:", ds_info)


Dataset Info: tfds.core.DatasetInfo(
    name='tf_flowers',
    full_name='tf_flowers/3.0.1',
    description="""
    A large set of images of flowers
    """,
    homepage='https://www.tensorflow.org/tutorials/load_data/images',
    data_path='/root/tensorflow_datasets/tf_flowers/3.0.1',
    file_format=tfrecord,
    download_size=218.21 MiB,
    dataset_size=221.83 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=5),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=3670, num_shards=2>,
    },
    citation="""@ONLINE {tfflowers,
    author = "The TensorFlow Team",
    title = "Flowers",
    month = "jan",
    year = "2019",
    url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }""",
)


### Step 2: Preprocess Data and Augmentation:

In [ ]:
# Image size and batch size
img_size = (299, 299)
batch_size = 32

# Preprocess function
def preprocess(image, label):
    image = tf.image.resize(image, img_size)
    image = tf.keras.applications.inception_v3.preprocess_input(image)
    return image, label

# Augmentation function
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

# Apply preprocessing and augmentation to training and validation datasets
ds_train = ds_train.map(preprocess).map(lambda x, y: (data_augmentation(x, training=True), y)).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
ds_validation = ds_validation.map(preprocess).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)


### Step 3: Load Pretrained InceptionV3 Model:

In [ ]:
# Load pre-trained InceptionV3 model with weights from ImageNet
base_model = tf.keras.applications.InceptionV3(input_shape=(img_size[0], img_size[1], 3),
                                               include_top=False,
                                               weights='imagenet')

# Freeze base layers
base_model.trainable = False

### Step 4: Build and Compile the Transfer Learning Model:


In [ ]:
# Add custom classification head
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(ds_info.features['label'].num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 22,854,437
Trainable params: 1,051,653
Non-trainable params: 21,802,784
__________________________________

### Step 5: Train and Evaluate the Model:


In [ ]:
# Train the model
epochs = 10
history = model.fit(ds_train, epochs=epochs, validation_data=ds_validation)

# Evaluate the model
loss, accuracy = model.evaluate(ds_validation)
print(f"Validation loss: {loss:.4f}, Validation accuracy: {accuracy:.4f}")


Epoch 1/10
92/92 [==============================] - 931s 10s/step - loss: 0.8001 - accuracy: 0.7013 - val_loss: 0.3877 - val_accuracy: 0.8692
Epoch 2/10
92/92 [==============================] - 891s 10s/step - loss: 0.4615 - accuracy: 0.8358 - val_loss: 0.3185 - val_accuracy: 0.9005
Epoch 3/10
92/92 [==============================] - 897s 10s/step - loss: 0.4120 - accuracy: 0.8522 - val_loss: 0.3145 - val_accuracy: 0.8978
Epoch 4/10
92/92 [==============================] - 894s 10s/step - loss: 0.3800 - accuracy: 0.8607 - val_loss: 0.2946 - val_accuracy: 0.9019
Epoch 5/10
92/92 [==============================] - 923s 10s/step - loss: 0.3520 - accuracy: 0.8753 - val_loss: 0.2831 - val_accuracy: 0.9033
Epoch 6/10
92/92 [==============================] - 923s 10s/step - loss: 0.3169 - accuracy: 0.8876 - val_loss: 0.2940 - val_accuracy: 0.8978
Epoch 7/10
92/92 [==============================] - 923s 10s/step - loss: 0.3090 - accuracy: 0.8808 - val_loss: 0.2708 - val_accuracy: 0.9046
Epoch 